In [ ]:
# swarm algorithms imports
from swarm_algo.firefly import Firefly as FA

# problem imports
from problem.knapsack import Knapsack
from problem.tsp import TSP
from problem.ackley import AckleyFunction

# visualization imports
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# utility imports
import random
import time
import logging
import os
from tqdm import tqdm
from typing import List, Tuple, Dict, Any
from pathlib import Path

In [ ]:
DATASET_FOLDER = str(Path.cwd() / ".." / "data")

In [ ]:
# Cấu hình logging và random seed
logging.basicConfig(level=logging.INFO)
np.random.seed(42)
random.seed(42)

# Thiết lập style cho visualization
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 10

# Firefly Algorithm (FA)

Các tham số quan trọng trong FA:
- `num_fireflies`: số lượng đom đóm sử dụng.
- `beta`: độ sáng (tầm nhìn) của đom đóm (giá trị từ 0 --> 1).
- `gamma`: mức độ suy giảm tầm nhìn dựa trên khoảng cách.
- `alpha`: hệ số chuyển động của đom đóm.

## Knapsack Problem

In [ ]:
def run_fa_knapsack(
    problem_num,
    num_fireflies=20,
    beta=1.0,
    gamma=1.0,
    alpha=0.2,
    max_generations=100,
    num_runs=5,
):
    knapsack_folder = os.path.join(DATASET_FOLDER, "knapsack")
    knapsack = Knapsack(PROBLEM_FOLDER=knapsack_folder, PROBLEM=problem_num)

    best_fitnesses = []
    convergence_histories = []
    execution_times = []

    for run in range(num_runs):
        # Khởi tạo FA
        fa = FA(
            ndim=len(knapsack.items),
            num_fireflies=num_fireflies,
            beta=beta,
            gamma=gamma,
            alpha=alpha,
            problem_type="binary",
        )

        # Set objective function
        fa.set_objective_function(knapsack.calculate_fitness)

        # Chạy thuật toán
        start_time = time.time()
        best_solution, best_fitness, history = fa.run(
            max_generations=max_generations, visualize=False
        )
        end_time = time.time()

        best_fitnesses.append(best_fitness)
        convergence_histories.append(history)
        execution_times.append(end_time - start_time)

    return {
        "best_fitness": np.max(best_fitnesses),
        "avg_fitness": np.mean(best_fitnesses),
        "std_fitness": np.std(best_fitnesses),
        "convergence_history": convergence_histories,
        "avg_time": np.mean(execution_times),
        "all_best_fitnesses": best_fitnesses,
    }

### 1. Phân tích độ nhạy của tham số `num_fireflies`

In [ ]:
# Test với các giá trị khác nhau của num_fireflies
problem_num = 1  # Sử dụng bài toán Knapsack số 1
num_fireflies_values = [5, 10, 15, 20, 30, 40, 50]
max_generations = 100
num_runs = 5

results_num_fireflies = []

print("Đang test tham số num_fireflies...")
for num_ff in tqdm(num_fireflies_values):
    result = run_fa_knapsack(
        problem_num=problem_num,
        num_fireflies=num_ff,
        beta=1.0,
        gamma=1.0,
        alpha=0.2,
        max_generations=max_generations,
        num_runs=num_runs,
    )
    result["num_fireflies"] = num_ff
    results_num_fireflies.append(result)
    print(
        f"num_fireflies={num_ff}: Best={result['best_fitness']:.2f}, Avg={result['avg_fitness']:.2f}, Time={result['avg_time']:.3f}s"
    )

In [ ]:
# Visualization cho num_fireflies
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Best Fitness vs num_fireflies
ax1 = axes[0, 0]
num_ff_vals = [r["num_fireflies"] for r in results_num_fireflies]
best_vals = [r["best_fitness"] for r in results_num_fireflies]
avg_vals = [r["avg_fitness"] for r in results_num_fireflies]
std_vals = [r["std_fitness"] for r in results_num_fireflies]

ax1.plot(num_ff_vals, best_vals, "bo-", linewidth=2, markersize=8, label="Best Fitness")
ax1.plot(
    num_ff_vals, avg_vals, "rs--", linewidth=2, markersize=8, label="Average Fitness"
)
ax1.fill_between(
    num_ff_vals,
    np.array(avg_vals) - np.array(std_vals),
    np.array(avg_vals) + np.array(std_vals),
    alpha=0.3,
    color="red",
    label="Std Dev",
)
ax1.set_xlabel("Số lượng đom đóm (num_fireflies)", fontsize=12, fontweight="bold")
ax1.set_ylabel("Fitness Value", fontsize=12, fontweight="bold")
ax1.set_title(
    "Ảnh hưởng của num_fireflies đến hiệu suất", fontsize=14, fontweight="bold"
)
ax1.legend(loc="best")
ax1.grid(True, alpha=0.3)

# 2. Execution Time vs num_fireflies
ax2 = axes[0, 1]
time_vals = [r["avg_time"] for r in results_num_fireflies]
ax2.bar(num_ff_vals, time_vals, color="skyblue", edgecolor="navy", alpha=0.7)
ax2.set_xlabel("Số lượng đom đóm (num_fireflies)", fontsize=12, fontweight="bold")
ax2.set_ylabel("Thời gian thực thi (s)", fontsize=12, fontweight="bold")
ax2.set_title("Thời gian thực thi theo num_fireflies", fontsize=14, fontweight="bold")
ax2.grid(True, alpha=0.3, axis="y")

# 3. Convergence curves for different num_fireflies
ax3 = axes[1, 0]
colors = plt.cm.viridis(np.linspace(0, 1, len(results_num_fireflies)))
for i, result in enumerate(results_num_fireflies):
    # Tính trung bình của các lần chạy
    avg_history = np.mean(result["convergence_history"], axis=0)
    ax3.plot(
        avg_history,
        color=colors[i],
        linewidth=2,
        label=f"num_fireflies={result['num_fireflies']}",
    )
ax3.set_xlabel("Generation", fontsize=12, fontweight="bold")
ax3.set_ylabel("Best Fitness", fontsize=12, fontweight="bold")
ax3.set_title(
    "Đường hội tụ với các giá trị num_fireflies", fontsize=14, fontweight="bold"
)
ax3.legend(loc="best", fontsize=9)
ax3.grid(True, alpha=0.3)

# 4. Box plot for variance analysis
ax4 = axes[1, 1]
data_for_boxplot = [r["all_best_fitnesses"] for r in results_num_fireflies]
bp = ax4.boxplot(data_for_boxplot, labels=num_ff_vals, patch_artist=True)
for patch, color in zip(bp["boxes"], colors):
    patch.set_facecolor(color)
ax4.set_xlabel("Số lượng đom đóm (num_fireflies)", fontsize=12, fontweight="bold")
ax4.set_ylabel("Best Fitness Distribution", fontsize=12, fontweight="bold")
ax4.set_title("Phân phối fitness theo num_fireflies", fontsize=14, fontweight="bold")
ax4.grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.show()

## 2. Phân tích độ nhạy của tham số `beta` (Attractiveness Coefficient)

In [ ]:
# Test với các giá trị khác nhau của beta
beta_values = [0.1, 0.3, 0.5, 0.7, 1.0, 1.5, 2.0]
results_beta = []

print("Đang test tham số beta...")
for beta in tqdm(beta_values):
    result = run_fa_knapsack(
        problem_num=problem_num,
        num_fireflies=20,
        beta=beta,
        gamma=1.0,
        alpha=0.2,
        max_generations=max_generations,
        num_runs=num_runs,
    )
    result["beta"] = beta
    results_beta.append(result)
    print(
        f"beta={beta}: Best={result['best_fitness']:.2f}, Avg={result['avg_fitness']:.2f}"
    )

In [ ]:
# Visualization cho beta
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Best Fitness vs beta
ax1 = axes[0, 0]
beta_vals = [r["beta"] for r in results_beta]
best_vals = [r["best_fitness"] for r in results_beta]
avg_vals = [r["avg_fitness"] for r in results_beta]
std_vals = [r["std_fitness"] for r in results_beta]

ax1.plot(beta_vals, best_vals, "go-", linewidth=2, markersize=8, label="Best Fitness")
ax1.plot(
    beta_vals, avg_vals, "ms--", linewidth=2, markersize=8, label="Average Fitness"
)
ax1.fill_between(
    beta_vals,
    np.array(avg_vals) - np.array(std_vals),
    np.array(avg_vals) + np.array(std_vals),
    alpha=0.3,
    color="magenta",
    label="Std Dev",
)
ax1.set_xlabel("Beta (Hệ số hấp dẫn)", fontsize=12, fontweight="bold")
ax1.set_ylabel("Fitness Value", fontsize=12, fontweight="bold")
ax1.set_title("Ảnh hưởng của beta đến hiệu suất", fontsize=14, fontweight="bold")
ax1.legend(loc="best")
ax1.grid(True, alpha=0.3)

# 2. Convergence curves for different beta
ax2 = axes[0, 1]
colors = plt.cm.plasma(np.linspace(0, 1, len(results_beta)))
for i, result in enumerate(results_beta):
    avg_history = np.mean(result["convergence_history"], axis=0)
    ax2.plot(avg_history, color=colors[i], linewidth=2, label=f"beta={result['beta']}")
ax2.set_xlabel("Generation", fontsize=12, fontweight="bold")
ax2.set_ylabel("Best Fitness", fontsize=12, fontweight="bold")
ax2.set_title("Đường hội tụ với các giá trị beta", fontsize=14, fontweight="bold")
ax2.legend(loc="best", fontsize=9)
ax2.grid(True, alpha=0.3)

# 3. Box plot for variance analysis
ax3 = axes[1, 0]
data_for_boxplot = [r["all_best_fitnesses"] for r in results_beta]
bp = ax3.boxplot(data_for_boxplot, labels=beta_vals, patch_artist=True)
for patch, color in zip(bp["boxes"], colors):
    patch.set_facecolor(color)
ax3.set_xlabel("Beta (Hệ số hấp dẫn)", fontsize=12, fontweight="bold")
ax3.set_ylabel("Best Fitness Distribution", fontsize=12, fontweight="bold")
ax3.set_title("Phân phối fitness theo beta", fontsize=14, fontweight="bold")
ax3.grid(True, alpha=0.3, axis="y")

# 4. Convergence speed (generations to reach threshold)
ax4 = axes[1, 1]
threshold = 0.9 * max([r["best_fitness"] for r in results_beta])
convergence_speeds = []
for result in results_beta:
    avg_history = np.mean(result["convergence_history"], axis=0)
    gen_to_threshold = (
        np.argmax(avg_history >= threshold)
        if np.any(avg_history >= threshold)
        else max_generations
    )
    convergence_speeds.append(gen_to_threshold)

ax4.bar(beta_vals, convergence_speeds, color="coral", edgecolor="darkred", alpha=0.7)
ax4.set_xlabel("Beta (Hệ số hấp dẫn)", fontsize=12, fontweight="bold")
ax4.set_ylabel("Generations to 90% best fitness", fontsize=12, fontweight="bold")
ax4.set_title("Tốc độ hội tụ theo beta", fontsize=14, fontweight="bold")
ax4.grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.show()

## 3. Phân tích độ nhạy của tham số `gamma` (Light Absorption Coefficient)

In [ ]:
# Test với các giá trị khác nhau của gamma
gamma_values = [0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
results_gamma = []

print("Đang test tham số gamma...")
for gamma in tqdm(gamma_values):
    result = run_fa_knapsack(
        problem_num=problem_num,
        num_fireflies=20,
        beta=1.0,
        gamma=gamma,
        alpha=0.2,
        max_generations=max_generations,
        num_runs=num_runs,
    )
    result["gamma"] = gamma
    results_gamma.append(result)
    print(
        f"gamma={gamma}: Best={result['best_fitness']:.2f}, Avg={result['avg_fitness']:.2f}"
    )

In [ ]:
# Visualization cho gamma
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Best Fitness vs gamma
ax1 = axes[0, 0]
gamma_vals = [r["gamma"] for r in results_gamma]
best_vals = [r["best_fitness"] for r in results_gamma]
avg_vals = [r["avg_fitness"] for r in results_gamma]
std_vals = [r["std_fitness"] for r in results_gamma]

ax1.semilogx(
    gamma_vals, best_vals, "co-", linewidth=2, markersize=8, label="Best Fitness"
)
ax1.semilogx(
    gamma_vals, avg_vals, "ys--", linewidth=2, markersize=8, label="Average Fitness"
)
ax1.fill_between(
    gamma_vals,
    np.array(avg_vals) - np.array(std_vals),
    np.array(avg_vals) + np.array(std_vals),
    alpha=0.3,
    color="yellow",
    label="Std Dev",
)
ax1.set_xlabel("Gamma (Hệ số hấp thụ ánh sáng)", fontsize=12, fontweight="bold")
ax1.set_ylabel("Fitness Value", fontsize=12, fontweight="bold")
ax1.set_title("Ảnh hưởng của gamma đến hiệu suất", fontsize=14, fontweight="bold")
ax1.legend(loc="best")
ax1.grid(True, alpha=0.3)

# 2. Convergence curves for different gamma
ax2 = axes[0, 1]
colors = plt.cm.cool(np.linspace(0, 1, len(results_gamma)))
for i, result in enumerate(results_gamma):
    avg_history = np.mean(result["convergence_history"], axis=0)
    ax2.plot(
        avg_history, color=colors[i], linewidth=2, label=f"gamma={result['gamma']}"
    )
ax2.set_xlabel("Generation", fontsize=12, fontweight="bold")
ax2.set_ylabel("Best Fitness", fontsize=12, fontweight="bold")
ax2.set_title("Đường hội tụ với các giá trị gamma", fontsize=14, fontweight="bold")
ax2.legend(loc="best", fontsize=9)
ax2.grid(True, alpha=0.3)

# 3. Box plot for variance analysis
ax3 = axes[1, 0]
data_for_boxplot = [r["all_best_fitnesses"] for r in results_gamma]
bp = ax3.boxplot(
    data_for_boxplot, labels=[f"{g}" for g in gamma_vals], patch_artist=True
)
for patch, color in zip(bp["boxes"], colors):
    patch.set_facecolor(color)
ax3.set_xlabel("Gamma (Hệ số hấp thụ ánh sáng)", fontsize=12, fontweight="bold")
ax3.set_ylabel("Best Fitness Distribution", fontsize=12, fontweight="bold")
ax3.set_title("Phân phối fitness theo gamma", fontsize=14, fontweight="bold")
ax3.grid(True, alpha=0.3, axis="y")
ax3.tick_params(axis="x", rotation=45)

# 4. Standard deviation analysis
ax4 = axes[1, 1]
ax4.plot(gamma_vals, std_vals, "ro-", linewidth=2, markersize=8)
ax4.set_xscale("log")
ax4.set_xlabel("Gamma (Hệ số hấp thụ ánh sáng)", fontsize=12, fontweight="bold")
ax4.set_ylabel("Standard Deviation", fontsize=12, fontweight="bold")
ax4.set_title("Độ ổn định theo gamma", fontsize=14, fontweight="bold")
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Phân tích độ nhạy của tham số `alpha` (Randomization Parameter)

In [ ]:
# Test với các giá trị khác nhau của alpha
alpha_values = [0.05, 0.1, 0.2, 0.3, 0.5, 0.7, 1.0]
results_alpha = []

print("Đang test tham số alpha...")
for alpha in tqdm(alpha_values):
    result = run_fa_knapsack(
        problem_num=problem_num,
        num_fireflies=20,
        beta=1.0,
        gamma=1.0,
        alpha=alpha,
        max_generations=max_generations,
        num_runs=num_runs,
    )
    result["alpha"] = alpha
    results_alpha.append(result)
    print(
        f"alpha={alpha}: Best={result['best_fitness']:.2f}, Avg={result['avg_fitness']:.2f}"
    )

In [ ]:
# Visualization cho alpha
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Best Fitness vs alpha
ax1 = axes[0, 0]
alpha_vals = [r["alpha"] for r in results_alpha]
best_vals = [r["best_fitness"] for r in results_alpha]
avg_vals = [r["avg_fitness"] for r in results_alpha]
std_vals = [r["std_fitness"] for r in results_alpha]

ax1.plot(alpha_vals, best_vals, "ko-", linewidth=2, markersize=8, label="Best Fitness")
ax1.plot(
    alpha_vals, avg_vals, "bs--", linewidth=2, markersize=8, label="Average Fitness"
)
ax1.fill_between(
    alpha_vals,
    np.array(avg_vals) - np.array(std_vals),
    np.array(avg_vals) + np.array(std_vals),
    alpha=0.3,
    color="blue",
    label="Std Dev",
)
ax1.set_xlabel("Alpha (Tham số ngẫu nhiên hóa)", fontsize=12, fontweight="bold")
ax1.set_ylabel("Fitness Value", fontsize=12, fontweight="bold")
ax1.set_title("Ảnh hưởng của alpha đến hiệu suất", fontsize=14, fontweight="bold")
ax1.legend(loc="best")
ax1.grid(True, alpha=0.3)

# 2. Convergence curves for different alpha
ax2 = axes[0, 1]
colors = plt.cm.autumn(np.linspace(0, 1, len(results_alpha)))
for i, result in enumerate(results_alpha):
    avg_history = np.mean(result["convergence_history"], axis=0)
    ax2.plot(
        avg_history, color=colors[i], linewidth=2, label=f"alpha={result['alpha']}"
    )
ax2.set_xlabel("Generation", fontsize=12, fontweight="bold")
ax2.set_ylabel("Best Fitness", fontsize=12, fontweight="bold")
ax2.set_title("Đường hội tụ với các giá trị alpha", fontsize=14, fontweight="bold")
ax2.legend(loc="best", fontsize=9)
ax2.grid(True, alpha=0.3)

# 3. Box plot for variance analysis
ax3 = axes[1, 0]
data_for_boxplot = [r["all_best_fitnesses"] for r in results_alpha]
bp = ax3.boxplot(data_for_boxplot, labels=alpha_vals, patch_artist=True)
for patch, color in zip(bp["boxes"], colors):
    patch.set_facecolor(color)
ax3.set_xlabel("Alpha (Tham số ngẫu nhiên hóa)", fontsize=12, fontweight="bold")
ax3.set_ylabel("Best Fitness Distribution", fontsize=12, fontweight="bold")
ax3.set_title("Phân phối fitness theo alpha", fontsize=14, fontweight="bold")
ax3.grid(True, alpha=0.3, axis="y")

# 4. Exploration vs Exploitation trade-off
ax4 = axes[1, 1]
# Tính độ thay đổi trung bình của fitness qua các thế hệ (đo lường exploration)
exploration_metric = []
for result in results_alpha:
    avg_history = np.mean(result["convergence_history"], axis=0)
    # Tính tổng sự thay đổi tuyệt đối
    changes = np.abs(np.diff(avg_history))
    exploration_metric.append(np.mean(changes))

ax4.plot(alpha_vals, exploration_metric, "go-", linewidth=2, markersize=10)
ax4.set_xlabel("Alpha (Tham số ngẫu nhiên hóa)", fontsize=12, fontweight="bold")
ax4.set_ylabel("Mức độ exploration (avg change)", fontsize=12, fontweight="bold")
ax4.set_title("Khả năng exploration theo alpha", fontsize=14, fontweight="bold")
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Phân tích tổng hợp (Heatmap) - Tương tác giữa các tham số

In [ ]:
# Phân tích tương tác giữa beta và gamma
beta_test = [0.3, 0.7, 1.0, 1.5, 2.0]
gamma_test = [0.1, 0.5, 1.0, 2.0, 5.0]

heatmap_data = np.zeros((len(beta_test), len(gamma_test)))

print("Đang test tương tác beta-gamma...")
for i, beta in enumerate(tqdm(beta_test)):
    for j, gamma in enumerate(gamma_test):
        result = run_fa_knapsack(
            problem_num=problem_num,
            num_fireflies=20,
            beta=beta,
            gamma=gamma,
            alpha=0.2,
            max_generations=max_generations,
            num_runs=3,  # Giảm số lần chạy để tăng tốc
        )
        heatmap_data[i, j] = result["avg_fitness"]
        print(f"beta={beta}, gamma={gamma}: Avg={result['avg_fitness']:.2f}")

In [ ]:
# Visualization heatmap cho tương tác beta-gamma
fig, ax = plt.subplots(figsize=(12, 8))

im = ax.imshow(heatmap_data, cmap="YlOrRd", aspect="auto")

# Thiết lập ticks và labels
ax.set_xticks(np.arange(len(gamma_test)))
ax.set_yticks(np.arange(len(beta_test)))
ax.set_xticklabels(gamma_test)
ax.set_yticklabels(beta_test)

# Thêm colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label("Average Fitness", fontsize=12, fontweight="bold")

# Thêm annotations
for i in range(len(beta_test)):
    for j in range(len(gamma_test)):
        text = ax.text(
            j,
            i,
            f"{heatmap_data[i, j]:.1f}",
            ha="center",
            va="center",
            color="black",
            fontsize=10,
        )

ax.set_xlabel("Gamma (Hệ số hấp thụ ánh sáng)", fontsize=12, fontweight="bold")
ax.set_ylabel("Beta (Hệ số hấp dẫn)", fontsize=12, fontweight="bold")
ax.set_title(
    "Tương tác giữa Beta và Gamma - Heatmap của Average Fitness",
    fontsize=14,
    fontweight="bold",
)

plt.tight_layout()
plt.show()

## Ackley Function Problem

In [ ]:
def run_fa_ackley(
    problem_num,
    num_fireflies=20,
    beta=1.0,
    gamma=1.0,
    alpha=0.2,
    max_generations=100,
    num_runs=5,
):
    ackley_folder = os.path.join(DATASET_FOLDER, "ackley")
    ackley = AckleyFunction(PROBLEM_FOLDER=ackley_folder, PROBLEM=problem_num)

    best_fitnesses = []
    convergence_histories = []
    execution_times = []

    for run in range(num_runs):
        # Khởi tạo FA
        fa = FA(
            ndim=ackley.dimension,
            num_fireflies=num_fireflies,
            beta=beta,
            gamma=gamma,
            alpha=alpha,
            problem_type="continuous",
        )

        # Set objective function (chú ý: Ackley tìm min nên cần negate hoặc điều chỉnh)
        # Firefly tìm max, nên ta cần convert bằng cách negate fitness
        def fitness_func(x):
            return -ackley.calculate_fitness(x)  # Negate vì FA tìm max

        fa.set_objective_function(fitness_func)

        # Chạy thuật toán
        start_time = time.time()
        best_solution, best_fitness, history = fa.run(
            max_generations=max_generations, visualize=False
        )
        end_time = time.time()

        # Convert lại fitness về giá trị thực (không negate)
        best_fitnesses.append(-best_fitness)
        convergence_histories.append([-h for h in history])
        execution_times.append(end_time - start_time)

    return {
        "best_fitness": np.min(best_fitnesses),  # Min vì Ackley tìm min
        "avg_fitness": np.mean(best_fitnesses),
        "std_fitness": np.std(best_fitnesses),
        "convergence_history": convergence_histories,
        "avg_time": np.mean(execution_times),
        "all_best_fitnesses": best_fitnesses,
    }

### Visualization: Ackley Function Landscape (3D)

In [ ]:
# Vẽ 3D Surface của Ackley Function để hiểu landscape
from mpl_toolkits.mplot3d import Axes3D


def ackley_function_2d(x, y):
    """Ackley function cho 2D visualization"""
    a = 20
    b = 0.2
    c = 2 * np.pi

    term1 = -a * np.exp(-b * np.sqrt((x**2 + y**2) / 2))
    term2 = -np.exp((np.cos(c * x) + np.cos(c * y)) / 2)
    result = term1 + term2 + a + np.e

    return result


# Tạo grid cho visualization
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(x, y)
Z = ackley_function_2d(X, Y)

# Tạo figure với 2 views
fig = plt.figure(figsize=(16, 6))

# View 1: Standard 3D surface
ax1 = fig.add_subplot(121, projection="3d")
surf1 = ax1.plot_surface(X, Y, Z, cmap="viridis", alpha=0.9, edgecolor="none")
ax1.contour(X, Y, Z, levels=20, cmap="viridis", linewidths=1, offset=0)

ax1.set_xlabel("X₁", fontsize=12, fontweight="bold")
ax1.set_ylabel("X₂", fontsize=12, fontweight="bold")
ax1.set_zlabel("f(X₁, X₂)", fontsize=12, fontweight="bold")
ax1.set_title(
    "Ackley Function Landscape\n(Standard View)", fontsize=13, fontweight="bold"
)
ax1.view_init(elev=30, azim=45)
fig.colorbar(surf1, ax=ax1, shrink=0.5, aspect=10, label="Fitness Value")

# View 2: Top-down view (Contour)
ax2 = fig.add_subplot(122, projection="3d")
surf2 = ax2.plot_surface(X, Y, Z, cmap="plasma", alpha=0.7, edgecolor="none")

# Thêm contour ở các mức khác nhau
ax2.contour(X, Y, Z, levels=30, cmap="plasma", linewidths=1.5, offset=0)

ax2.set_xlabel("X₁", fontsize=12, fontweight="bold")
ax2.set_ylabel("X₂", fontsize=12, fontweight="bold")
ax2.set_zlabel("f(X₁, X₂)", fontsize=12, fontweight="bold")
ax2.set_title(
    "Ackley Function Landscape\n(Top-Down View)", fontsize=13, fontweight="bold"
)
ax2.view_init(elev=60, azim=45)  # Top-down view
fig.colorbar(surf2, ax=ax2, shrink=0.5, aspect=10, label="Fitness Value")

plt.tight_layout()
plt.show()

print("\n🔍 Quan sát về Ackley Function:")
print("- Global minimum tại (0, 0) với f(0,0) = 0")
print("- Có RẤT NHIỀU local minima (các vùng lõm nhỏ)")
print("- Bề mặt có cấu trúc 'egg-crate' với nhiều đỉnh và thung lũng")
print("- Đây là lý do tại sao cần:")
print("  + Beta thấp (0.7-1.0): tránh hội tụ sớm vào local minima")
print("  + Gamma thấp (0.5-1.0): duy trì exploration để tìm global minimum")
print("  + Alpha vừa phải (0.2-0.3): đủ random để thoát khỏi local minima")

### Visualization: Convergence Trajectory trên Ackley Landscape

In [ ]:
# Chạy FA và track TOÀN BỘ SWARM qua từng generation
print("🔄 Đang chạy FA để track trajectory của toàn bộ swarm...")

# Load Ackley problem
ackley_folder = os.path.join(DATASET_FOLDER, "ackley")
ackley = AckleyFunction(PROBLEM_FOLDER=ackley_folder, PROBLEM=1)

# Khởi tạo FA với tham số tối ưu
fa = FA(
    ndim=2,  # Chỉ dùng 2D để visualize
    num_fireflies=20,
    beta=0.8,
    gamma=0.7,
    alpha=0.25,
    problem_type="continuous",
)

# Track history của TOÀN BỘ swarm
all_fireflies_history = []  # Lưu vị trí tất cả fireflies qua các gen
all_fitness_history = []  # Lưu fitness tất cả fireflies qua các gen
best_solution_history = []  # Lưu best solution của mỗi gen
best_fitness_history = []  # Lưu best fitness của mỗi gen


# Setup objective function
def tracked_fitness(x):
    fitness = ackley.calculate_fitness(x)
    return -fitness  # Negate vì FA maximize


fa.set_objective_function(tracked_fitness)

# Initialize swarm
np.random.seed(42)
fa.positions = np.random.uniform(-5, 5, (fa.num_fireflies, 2))
fa.intensities = np.array([tracked_fitness(f) for f in fa.positions])

print(f"🔍 Initial best fitness: {-np.max(fa.intensities):.4f}")

# Run và track qua từng generation
num_generations = 50
for gen in range(num_generations):
    # Save current state
    all_fireflies_history.append(fa.positions.copy())
    all_fitness_history.append(fa.intensities.copy())

    # Track best của generation này
    best_idx = np.argmax(fa.intensities)
    best_solution_history.append(fa.positions[best_idx].copy())
    best_fitness_history.append(-fa.intensities[best_idx])  # Convert back to minimize

    # Update positions cho generation kế tiếp
    if gen < num_generations - 1:  # Không update ở generation cuối
        fa.update_positions()

        # ✅✅✅ CRITICAL: PHẢI recalculate fitness sau khi update positions! ✅✅✅
        # Nếu thiếu dòng này, fireflies di chuyển nhưng fitness không đổi!
        fa.intensities = np.array([tracked_fitness(f) for f in fa.positions])

        # Debug every 10 gens
        if (gen + 1) % 10 == 0:
            current_best = -np.max(fa.intensities)
            improvement = best_fitness_history[0] - current_best
            print(
                f"   Gen {gen+1:2d}: Best = {current_best:.4f} | Improvement = {improvement:.4f}"
            )

print(f"\n✅ Hoàn thành! Tracked {len(all_fireflies_history)} generations")
print(f"📍 Number of fireflies: {fa.num_fireflies}")
print(f"📍 Initial best fitness: {best_fitness_history[0]:.4f}")
print(f"📍 Final best fitness: {best_fitness_history[-1]:.4f}")
print(f"📍 Total improvement: {best_fitness_history[0] - best_fitness_history[-1]:.4f}")
print(f"📍 Distance to optimum (0,0): {np.linalg.norm(best_solution_history[-1]):.4f}")

if best_fitness_history[0] - best_fitness_history[-1] < 0.01:
    print("\n⚠️  WARNING: Fitness không improve! Có thể thiếu recalculate fitness!")
else:
    print("\n✅ SUCCESS: Fitness đang được update đúng!")

In [ ]:
# 🔍 Verification: Check convergence behavior
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot 1: Best fitness over generations
ax1 = axes[0]
ax1.plot(best_fitness_history, "b-", linewidth=2, marker="o", markersize=3)
ax1.axhline(y=0, color="red", linestyle="--", linewidth=1, label="Global Optimum")
ax1.set_xlabel("Generation", fontweight="bold")
ax1.set_ylabel("Best Fitness", fontweight="bold")
ax1.set_title("Best Fitness Convergence", fontweight="bold")
ax1.grid(True, alpha=0.3)
ax1.legend()

# Plot 2: Fitness improvement per generation
ax2 = axes[1]
improvements = [
    best_fitness_history[i] - best_fitness_history[i + 1]
    for i in range(len(best_fitness_history) - 1)
]
ax2.bar(
    range(len(improvements)), improvements, color="green", alpha=0.7, edgecolor="black"
)
ax2.set_xlabel("Generation", fontweight="bold")
ax2.set_ylabel("Fitness Improvement", fontweight="bold")
ax2.set_title("Per-Generation Improvement", fontweight="bold")
ax2.grid(True, alpha=0.3, axis="y")
ax2.axhline(y=0, color="red", linestyle="-", linewidth=1)

# Plot 3: Distance to optimum over time
ax3 = axes[2]
distances = [np.linalg.norm(sol) for sol in best_solution_history]
ax3.plot(distances, "r-", linewidth=2, marker="s", markersize=3)
ax3.axhline(y=0, color="green", linestyle="--", linewidth=1, label="Optimum (0,0)")
ax3.set_xlabel("Generation", fontweight="bold")
ax3.set_ylabel("Distance to Optimum", fontweight="bold")
ax3.set_title("Euclidean Distance to (0,0)", fontweight="bold")
ax3.grid(True, alpha=0.3)
ax3.legend()

plt.tight_layout()
plt.show()

# Statistics
print("\n📊 Convergence Statistics:")
print(
    f"   Total generations with improvement: {sum(1 for imp in improvements if imp > 1e-6)}/{len(improvements)}"
)
print(f"   Largest single improvement: {max(improvements):.6f}")
print(f"   Average improvement per gen: {np.mean(improvements):.6f}")
print(f"   Final distance to optimum: {distances[-1]:.6f}")
print(
    f"   Convergence rate (first half): {(best_fitness_history[0] - best_fitness_history[24])/25:.6f}"
)
print(
    f"   Convergence rate (second half): {(best_fitness_history[25] - best_fitness_history[-1])/25:.6f}"
)

In [ ]:
# Animation: Swarm movement qua từng generation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

print("🎬 Đang tạo animation...")

# Tạo landscape
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(x, y)
Z = ackley_function_2d(X, Y)

# Setup figure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Left plot: Contour với swarm
contour = ax1.contourf(X, Y, Z, levels=30, cmap="viridis", alpha=0.6)
ax1.contour(X, Y, Z, levels=30, cmap="viridis", linewidths=0.5, alpha=0.8)
plt.colorbar(contour, ax=ax1, label="Fitness Value")

# Initialize scatter plots
swarm_scatter = ax1.scatter(
    [],
    [],
    c="orange",
    s=100,
    marker="o",
    edgecolor="black",
    linewidth=1.5,
    alpha=0.7,
    label="Fireflies",
)
best_scatter = ax1.scatter(
    [],
    [],
    c="red",
    s=300,
    marker="*",
    edgecolor="black",
    linewidth=2,
    label="Best Firefly",
    zorder=10,
)
optimum_scatter = ax1.scatter(
    [0],
    [0],
    c="cyan",
    s=300,
    marker="X",
    edgecolor="black",
    linewidth=2,
    label="Global Optimum",
    zorder=9,
)

# Trajectory line (best solution path)
(trajectory_line,) = ax1.plot([], [], "r-", linewidth=2, alpha=0.5, label="Best Path")

ax1.set_xlabel("X₁", fontsize=12, fontweight="bold")
ax1.set_ylabel("X₂", fontsize=12, fontweight="bold")
ax1.set_xlim(-5, 5)
ax1.set_ylim(-5, 5)
ax1.legend(loc="upper right", fontsize=10)
ax1.grid(True, alpha=0.3)
title1 = ax1.set_title("Generation 0", fontsize=13, fontweight="bold")

# Right plot: Convergence curve
ax2.set_xlabel("Generation", fontsize=12, fontweight="bold")
ax2.set_ylabel("Best Fitness", fontsize=12, fontweight="bold")
ax2.set_title("Convergence Progress", fontsize=13, fontweight="bold")
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, len(best_fitness_history))
ax2.set_ylim(0, max(best_fitness_history) * 1.1)

(convergence_line,) = ax2.plot([], [], "b-", linewidth=2.5, label="Best Fitness")
current_point = ax2.scatter(
    [], [], c="red", s=150, marker="o", edgecolor="black", linewidth=2, zorder=10
)
ax2.axhline(y=0, color="cyan", linestyle="--", linewidth=2, label="Global Optimum")
ax2.legend(loc="upper right", fontsize=10)


# Animation function
def animate(frame):
    # Update swarm positions
    fireflies = all_fireflies_history[frame]
    swarm_scatter.set_offsets(fireflies)

    # Update best firefly
    best_pos = best_solution_history[frame]
    best_scatter.set_offsets([best_pos])

    # Update trajectory line (path of best solutions)
    trajectory_x = [sol[0] for sol in best_solution_history[: frame + 1]]
    trajectory_y = [sol[1] for sol in best_solution_history[: frame + 1]]
    trajectory_line.set_data(trajectory_x, trajectory_y)

    # Update title
    title1.set_text(
        f"Generation {frame} | Best Fitness: {best_fitness_history[frame]:.4f}"
    )

    # Update convergence curve
    convergence_line.set_data(range(frame + 1), best_fitness_history[: frame + 1])
    current_point.set_offsets([[frame, best_fitness_history[frame]]])

    return (
        swarm_scatter,
        best_scatter,
        trajectory_line,
        title1,
        convergence_line,
        current_point,
    )


# Create animation
anim = FuncAnimation(
    fig,
    animate,
    frames=len(all_fireflies_history),
    interval=200,
    blit=True,
    repeat=True,
)

plt.tight_layout()
print("✅ Animation ready! Displaying...")

# Display animation
HTML(anim.to_jshtml())

In [ ]:
# Static visualization: Snapshots tại các generation khác nhau
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# Chọn các generation để hiển thị
snapshot_gens = [0, 5, 10, 20, 35, 49]  # Gen đầu, giữa, cuối

for idx, gen in enumerate(snapshot_gens):
    ax = axes[idx]

    # Plot contour
    contour = ax.contourf(X, Y, Z, levels=20, cmap="viridis", alpha=0.5)
    ax.contour(X, Y, Z, levels=20, cmap="viridis", linewidths=0.5)

    # Plot trajectory up to this generation
    trajectory_x = [sol[0] for sol in best_solution_history[: gen + 1]]
    trajectory_y = [sol[1] for sol in best_solution_history[: gen + 1]]
    ax.plot(trajectory_x, trajectory_y, "r-", linewidth=2, alpha=0.6, label="Best Path")

    # Plot all fireflies at this generation
    fireflies = all_fireflies_history[gen]
    fitness_vals = -all_fitness_history[gen]  # Convert back to minimize

    scatter = ax.scatter(
        fireflies[:, 0],
        fireflies[:, 1],
        c=fitness_vals,
        cmap="hot",
        s=150,
        marker="o",
        edgecolor="black",
        linewidth=1.5,
        alpha=0.8,
        vmin=0,
        vmax=max(best_fitness_history),
    )

    # Highlight best firefly
    best_pos = best_solution_history[gen]
    ax.scatter(
        best_pos[0],
        best_pos[1],
        c="lime",
        s=400,
        marker="*",
        edgecolor="black",
        linewidth=2.5,
        label="Best",
        zorder=10,
    )

    # Mark global optimum
    ax.scatter(
        0,
        0,
        c="cyan",
        s=400,
        marker="X",
        edgecolor="black",
        linewidth=2.5,
        label="Optimum",
        zorder=9,
    )

    ax.set_xlabel("X₁", fontsize=11, fontweight="bold")
    ax.set_ylabel("X₂", fontsize=11, fontweight="bold")
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    ax.set_title(
        f"Generation {gen}\nBest Fitness: {best_fitness_history[gen]:.4f}",
        fontsize=12,
        fontweight="bold",
    )
    ax.legend(loc="upper right", fontsize=9)
    ax.grid(True, alpha=0.3)

    # Add colorbar
    plt.colorbar(scatter, ax=ax, label="Fitness")

plt.suptitle(
    "Firefly Algorithm Evolution Over Time", fontsize=16, fontweight="bold", y=1.00
)
plt.tight_layout()
plt.show()

# Statistics
print("\n📊 Phân tích Evolution của Swarm:")
print(f"- Initial spread: {np.std(all_fireflies_history[0]):.3f}")
print(f"- Final spread: {np.std(all_fireflies_history[-1]):.3f}")
print(
    f"- Convergence ratio: {np.std(all_fireflies_history[-1]) / np.std(all_fireflies_history[0]):.3f}"
)
print(
    f"- Final best distance to optimum: {np.linalg.norm(best_solution_history[-1]):.4f}"
)
print(
    f"- Average fitness improvement per gen: {(best_fitness_history[0] - best_fitness_history[-1]) / len(best_fitness_history):.5f}"
)

### 🎯 Kết luận về Convergence Behavior

**Quan sát từ Animation và Snapshots:**

1. **Exploration Phase (Gen 0-10):** 
   - Fireflies phân tán rộng khắp search space
   - Swarm tìm kiếm nhiều vùng khác nhau đồng thời
   - Best solution nhảy nhiều do phát hiện các vùng tốt hơn

2. **Transition Phase (Gen 10-25):**
   - Fireflies bắt đầu hội tụ về các vùng có fitness tốt
   - Một số fireflies vẫn explore ở xa để tránh local minima
   - Best solution ổn định hơn, di chuyển từ từ về phía optimum

3. **Exploitation Phase (Gen 25-50):**
   - Swarm tập trung quanh global optimum (0, 0)
   - Các fireflies kém hơn bị hấp dẫn bởi fireflies tốt (attraction mechanism)
   - Best solution tiến gần đến (0, 0) với tốc độ chậm dần (fine-tuning)

**Điểm mạnh của FA thể hiện qua visualization:**
- ✅ **Balance tốt** giữa exploration và exploitation
- ✅ **Không bị stuck** ở local minima nhờ random walk (alpha)
- ✅ **Convergence ổn định** nhờ attraction mechanism (beta, gamma)
- ✅ **Swarm cooperation**: fireflies tốt hơn giúp dẫn đường cho fireflies kém hơn

In [ ]:
# Visualization: Trajectory overlay trên Ackley landscape
fig = plt.figure(figsize=(18, 6))

# Convert trajectory to arrays - FIX: best_solution_history (không có 's')
trajectory = np.array(best_solution_history)
x_path = trajectory[:, 0]
y_path = trajectory[:, 1]
z_path = np.array(best_fitness_history)

# Tạo landscape
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(x, y)
Z = ackley_function_2d(X, Y)

# --- Plot 1: 3D Trajectory ---
ax1 = fig.add_subplot(131, projection="3d")
surf = ax1.plot_surface(X, Y, Z, cmap="viridis", alpha=0.3, edgecolor="none")

# Vẽ trajectory
ax1.plot(x_path, y_path, z_path, "r-", linewidth=3, label="Search Path", zorder=10)
ax1.scatter(
    x_path[::5],
    y_path[::5],
    z_path[::5],
    c=range(0, len(x_path), 5),
    cmap="hot",
    s=50,
    marker="o",
    edgecolor="black",
    linewidth=1,
    zorder=11,
)

# Đánh dấu start và end
ax1.scatter(
    [x_path[0]],
    [y_path[0]],
    [z_path[0]],
    c="lime",
    s=200,
    marker="*",
    edgecolor="black",
    linewidth=2,
    label="Start",
    zorder=12,
)
ax1.scatter(
    [x_path[-1]],
    [y_path[-1]],
    [z_path[-1]],
    c="gold",
    s=200,
    marker="*",
    edgecolor="black",
    linewidth=2,
    label="End",
    zorder=12,
)
ax1.scatter(
    [0],
    [0],
    [0],
    c="cyan",
    s=200,
    marker="X",
    edgecolor="black",
    linewidth=2,
    label="Global Optimum",
    zorder=12,
)

ax1.set_xlabel("X₁", fontsize=11, fontweight="bold")
ax1.set_ylabel("X₂", fontsize=11, fontweight="bold")
ax1.set_zlabel("f(X)", fontsize=11, fontweight="bold")
ax1.set_title("3D Trajectory on Landscape", fontsize=12, fontweight="bold")
ax1.legend(loc="upper right", fontsize=9)
ax1.view_init(elev=25, azim=45)

# --- Plot 2: Top-down view (2D Contour với trajectory) ---
ax2 = fig.add_subplot(132)
contour = ax2.contour(X, Y, Z, levels=30, cmap="viridis", alpha=0.6)
ax2.contourf(X, Y, Z, levels=30, cmap="viridis", alpha=0.3)

# Vẽ trajectory với gradient color
for i in range(len(x_path) - 1):
    ax2.plot(x_path[i : i + 2], y_path[i : i + 2], "r-", linewidth=2, alpha=0.7)

# Đánh dấu các điểm theo generation
scatter = ax2.scatter(
    x_path[::2],
    y_path[::2],
    c=range(0, len(x_path), 2),
    cmap="hot",
    s=60,
    marker="o",
    edgecolor="black",
    linewidth=1,
    zorder=10,
)

ax2.scatter(
    x_path[0],
    y_path[0],
    c="lime",
    s=250,
    marker="*",
    edgecolor="black",
    linewidth=2,
    label="Start",
    zorder=11,
)
ax2.scatter(
    x_path[-1],
    y_path[-1],
    c="gold",
    s=250,
    marker="*",
    edgecolor="black",
    linewidth=2,
    label="End",
    zorder=11,
)
ax2.scatter(
    0,
    0,
    c="cyan",
    s=250,
    marker="X",
    edgecolor="black",
    linewidth=2,
    label="Global Optimum",
    zorder=11,
)

ax2.set_xlabel("X₁", fontsize=11, fontweight="bold")
ax2.set_ylabel("X₂", fontsize=11, fontweight="bold")
ax2.set_title("Top-Down View: Search Path", fontsize=12, fontweight="bold")
ax2.legend(loc="upper right", fontsize=9)
ax2.grid(True, alpha=0.3)
plt.colorbar(scatter, ax=ax2, label="Generation")

# --- Plot 3: Fitness convergence curve ---
ax3 = fig.add_subplot(133)
generations = np.arange(len(best_fitness_history))

ax3.plot(generations, best_fitness_history, "b-", linewidth=2.5, label="Best Fitness")
ax3.axhline(y=0, color="cyan", linestyle="--", linewidth=2, label="Global Optimum (0)")
ax3.fill_between(generations, best_fitness_history, 0, alpha=0.3, color="blue")

# Đánh dấu các milestone
milestones = [
    0,
    len(best_fitness_history) // 4,
    len(best_fitness_history) // 2,
    3 * len(best_fitness_history) // 4,
    len(best_fitness_history) - 1,
]
for m in milestones:
    ax3.scatter(
        m,
        best_fitness_history[m],
        s=100,
        c="red",
        edgecolor="black",
        linewidth=2,
        zorder=10,
    )
    ax3.annotate(
        f"Gen {m}\n{best_fitness_history[m]:.3f}",
        xy=(m, best_fitness_history[m]),
        xytext=(10, 10),
        textcoords="offset points",
        fontsize=8,
        bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.7),
        arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=0", lw=1),
    )

ax3.set_xlabel("Generation", fontsize=11, fontweight="bold")
ax3.set_ylabel("Best Fitness Value", fontsize=11, fontweight="bold")
ax3.set_title("Convergence Curve", fontsize=12, fontweight="bold")
ax3.legend(loc="upper right", fontsize=10)
ax3.grid(True, alpha=0.3)
ax3.set_yscale("log")

plt.tight_layout()
plt.show()

print("\n🎯 Phân tích Trajectory:")
print(
    f"- Quãng đường đi: {np.sum(np.sqrt(np.sum(np.diff(trajectory, axis=0)**2, axis=1))):.2f} units"
)
print(f"- Khoảng cách đến optimum: {np.sqrt(x_path[-1]**2 + y_path[-1]**2):.4f}")
print(
    f"- Tốc độ hội tụ: {(best_fitness_history[0] - best_fitness_history[-1])/len(best_fitness_history):.4f} per generation"
)

### 1. Phân tích độ nhạy của tham số `num_fireflies` cho Ackley

In [ ]:
# Test với các giá trị khác nhau của num_fireflies
problem_num_ackley = 1  # Sử dụng bài toán Ackley số 1
num_fireflies_values_ackley = [5, 10, 15, 20, 30, 40, 50]
max_generations_ackley = 100
num_runs_ackley = 5

results_num_fireflies_ackley = []

print("Đang test tham số num_fireflidung es cho Ackley Function...")
for num_ff in tqdm(num_fireflies_values_ackley):
    result = run_fa_ackley(
        problem_num=problem_num_ackley,
        num_fireflies=num_ff,
        beta=1.0,
        gamma=1.0,
        alpha=0.2,
        max_generations=max_generations_ackley,
        num_runs=num_runs_ackley,
    )
    result["num_fireflies"] = num_ff
    results_num_fireflies_ackley.append(result)
    print(
        f"num_fireflies={num_ff}: Best={result['best_fitness']:.4f}, Avg={result['avg_fitness']:.4f}, Time={result['avg_time']:.3f}s"
    )

In [ ]:
# Visualization cho num_fireflies - Ackley
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Best Fitness vs num_fireflies
ax1 = axes[0, 0]
num_ff_vals_ackley = [r["num_fireflies"] for r in results_num_fireflies_ackley]
best_vals_ackley = [r["best_fitness"] for r in results_num_fireflies_ackley]
avg_vals_ackley = [r["avg_fitness"] for r in results_num_fireflies_ackley]
std_vals_ackley = [r["std_fitness"] for r in results_num_fireflies_ackley]

ax1.plot(
    num_ff_vals_ackley,
    best_vals_ackley,
    "bo-",
    linewidth=2,
    markersize=8,
    label="Best Fitness",
)
ax1.plot(
    num_ff_vals_ackley,
    avg_vals_ackley,
    "rs--",
    linewidth=2,
    markersize=8,
    label="Average Fitness",
)
ax1.fill_between(
    num_ff_vals_ackley,
    np.array(avg_vals_ackley) - np.array(std_vals_ackley),
    np.array(avg_vals_ackley) + np.array(std_vals_ackley),
    alpha=0.3,
    color="red",
    label="Std Dev",
)
ax1.set_xlabel("Số lượng đom đóm (num_fireflies)", fontsize=12, fontweight="bold")
ax1.set_ylabel("Fitness Value (Ackley)", fontsize=12, fontweight="bold")
ax1.set_title(
    "Ảnh hưởng của num_fireflies đến hiệu suất (Ackley)", fontsize=14, fontweight="bold"
)
ax1.legend(loc="best")
ax1.grid(True, alpha=0.3)

# 2. Execution Time vs num_fireflies
ax2 = axes[0, 1]
time_vals_ackley = [r["avg_time"] for r in results_num_fireflies_ackley]
ax2.bar(
    num_ff_vals_ackley, time_vals_ackley, color="skyblue", edgecolor="navy", alpha=0.7
)
ax2.set_xlabel("Số lượng đom đóm (num_fireflies)", fontsize=12, fontweight="bold")
ax2.set_ylabel("Thời gian thực thi (s)", fontsize=12, fontweight="bold")
ax2.set_title(
    "Thời gian thực thi theo num_fireflies (Ackley)", fontsize=14, fontweight="bold"
)
ax2.grid(True, alpha=0.3, axis="y")

# 3. Convergence curves for different num_fireflies
ax3 = axes[1, 0]
colors = plt.cm.viridis(np.linspace(0, 1, len(results_num_fireflies_ackley)))
for i, result in enumerate(results_num_fireflies_ackley):
    # Tính trung bình của các lần chạy
    avg_history = np.mean(result["convergence_history"], axis=0)
    ax3.plot(
        avg_history,
        color=colors[i],
        linewidth=2,
        label=f"num_fireflies={result['num_fireflies']}",
    )
ax3.set_xlabel("Generation", fontsize=12, fontweight="bold")
ax3.set_ylabel("Best Fitness (Ackley)", fontsize=12, fontweight="bold")
ax3.set_title(
    "Đường hội tụ với các giá trị num_fireflies (Ackley)",
    fontsize=14,
    fontweight="bold",
)
ax3.legend(loc="best", fontsize=9)
ax3.grid(True, alpha=0.3)
ax3.set_yscale("log")  # Log scale for better visualization

# 4. Box plot for variance analysis
ax4 = axes[1, 1]
data_for_boxplot_ackley = [
    r["all_best_fitnesses"] for r in results_num_fireflies_ackley
]
bp = ax4.boxplot(data_for_boxplot_ackley, labels=num_ff_vals_ackley, patch_artist=True)
for patch, color in zip(bp["boxes"], colors):
    patch.set_facecolor(color)
ax4.set_xlabel("Số lượng đom đóm (num_fireflies)", fontsize=12, fontweight="bold")
ax4.set_ylabel("Best Fitness Distribution (Ackley)", fontsize=12, fontweight="bold")
ax4.set_title(
    "Phân phối fitness theo num_fireflies (Ackley)", fontsize=14, fontweight="bold"
)
ax4.grid(True, alpha=0.3, axis="y")
ax4.set_yscale("log")  # Log scale for better visualization

plt.tight_layout()
plt.show()

#### Kết luận cho tham số `num_fireflies`

**Quan sát:**
- Số lượng đom đóm có ảnh hưởng đáng kể đến hiệu suất tối ưu hóa Ackley Function
- Với num_fireflies thấp (5-10): Thuật toán hội tụ nhanh nhưng dễ bị kẹt ở local optima
- Với num_fireflies vừa phải (15-30): Đạt được cân bằng tốt giữa exploration và exploitation
- Với num_fireflies cao (40-50): Tăng khả năng tìm kiếm global optima nhưng tốn thời gian tính toán

**Khuyến nghị:**
- **Tốt nhất**: num_fireflies = 20-30 cho bài toán Ackley với dimension vừa phải
- Giá trị này cho phép thuật toán khám phá không gian tìm kiếm hiệu quả mà không tốn quá nhiều thời gian
- Đối với dimension cao hơn, nên tăng num_fireflies tương ứng (quy tắc: ~2-3 lần dimension)

### 2. Phân tích độ nhạy của tham số `beta` cho Ackley

In [ ]:
# Test với các giá trị khác nhau của beta
beta_values_ackley = [0.1, 0.3, 0.5, 0.7, 1.0, 1.5, 2.0]
results_beta_ackley = []

print("Đang test tham số beta cho Ackley...")
for beta in tqdm(beta_values_ackley):
    result = run_fa_ackley(
        problem_num=problem_num_ackley,
        num_fireflies=20,
        beta=beta,
        gamma=1.0,
        alpha=0.2,
        max_generations=max_generations_ackley,
        num_runs=num_runs_ackley,
    )
    result["beta"] = beta
    results_beta_ackley.append(result)
    print(
        f"beta={beta}: Best={result['best_fitness']:.4f}, Avg={result['avg_fitness']:.4f}"
    )

In [ ]:
# Visualization cho beta - Ackley
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Best Fitness vs beta
ax1 = axes[0, 0]
beta_vals_ackley = [r["beta"] for r in results_beta_ackley]
best_vals_ackley = [r["best_fitness"] for r in results_beta_ackley]
avg_vals_ackley = [r["avg_fitness"] for r in results_beta_ackley]
std_vals_ackley = [r["std_fitness"] for r in results_beta_ackley]

ax1.plot(
    beta_vals_ackley,
    best_vals_ackley,
    "go-",
    linewidth=2,
    markersize=8,
    label="Best Fitness",
)
ax1.plot(
    beta_vals_ackley,
    avg_vals_ackley,
    "ms--",
    linewidth=2,
    markersize=8,
    label="Average Fitness",
)
ax1.fill_between(
    beta_vals_ackley,
    np.array(avg_vals_ackley) - np.array(std_vals_ackley),
    np.array(avg_vals_ackley) + np.array(std_vals_ackley),
    alpha=0.3,
    color="magenta",
    label="Std Dev",
)
ax1.set_xlabel("Beta (Hệ số hấp dẫn)", fontsize=12, fontweight="bold")
ax1.set_ylabel("Fitness Value (Ackley)", fontsize=12, fontweight="bold")
ax1.set_title(
    "Ảnh hưởng của beta đến hiệu suất (Ackley)", fontsize=14, fontweight="bold"
)
ax1.legend(loc="best")
ax1.grid(True, alpha=0.3)
ax1.set_yscale("log")

# 2. Convergence curves for different beta
ax2 = axes[0, 1]
colors = plt.cm.plasma(np.linspace(0, 1, len(results_beta_ackley)))
for i, result in enumerate(results_beta_ackley):
    avg_history = np.mean(result["convergence_history"], axis=0)
    ax2.plot(avg_history, color=colors[i], linewidth=2, label=f"beta={result['beta']}")
ax2.set_xlabel("Generation", fontsize=12, fontweight="bold")
ax2.set_ylabel("Best Fitness (Ackley)", fontsize=12, fontweight="bold")
ax2.set_title(
    "Đường hội tụ với các giá trị beta (Ackley)", fontsize=14, fontweight="bold"
)
ax2.legend(loc="best", fontsize=9)
ax2.grid(True, alpha=0.3)
ax2.set_yscale("log")

# 3. Box plot for variance analysis
ax3 = axes[1, 0]
data_for_boxplot_ackley = [r["all_best_fitnesses"] for r in results_beta_ackley]
bp = ax3.boxplot(data_for_boxplot_ackley, labels=beta_vals_ackley, patch_artist=True)
for patch, color in zip(bp["boxes"], colors):
    patch.set_facecolor(color)
ax3.set_xlabel("Beta (Hệ số hấp dẫn)", fontsize=12, fontweight="bold")
ax3.set_ylabel("Best Fitness Distribution (Ackley)", fontsize=12, fontweight="bold")
ax3.set_title("Phân phối fitness theo beta (Ackley)", fontsize=14, fontweight="bold")
ax3.grid(True, alpha=0.3, axis="y")
ax3.set_yscale("log")

# 4. Convergence speed (generations to reach threshold)
ax4 = axes[1, 1]
threshold_ackley = 0.1  # Ngưỡng cho Ackley (gần optimal)
convergence_speeds_ackley = []
for result in results_beta_ackley:
    avg_history = np.mean(result["convergence_history"], axis=0)
    gen_to_threshold = (
        np.argmax(avg_history <= threshold_ackley)
        if np.any(avg_history <= threshold_ackley)
        else max_generations_ackley
    )
    convergence_speeds_ackley.append(gen_to_threshold)

ax4.bar(
    beta_vals_ackley,
    convergence_speeds_ackley,
    color="coral",
    edgecolor="darkred",
    alpha=0.7,
)
ax4.set_xlabel("Beta (Hệ số hấp dẫn)", fontsize=12, fontweight="bold")
ax4.set_ylabel("Generations to reach 0.1", fontsize=12, fontweight="bold")
ax4.set_title("Tốc độ hội tụ theo beta (Ackley)", fontsize=14, fontweight="bold")
ax4.grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.show()

#### Kết luận cho tham số `beta`

**Quan sát:**
- Beta (hệ số hấp dẫn) kiểm soát mức độ đom đóm di chuyển về phía đom đóm sáng hơn
- Beta thấp (0.1-0.3): Giảm tương tác giữa các đom đóm, tăng tính độc lập trong tìm kiếm
- Beta trung bình (0.5-1.0): Cân bằng tốt giữa tương tác và độc lập
- Beta cao (1.5-2.0): Tăng cường hội tụ nhưng có thể bị premature convergence

**Khuyến nghị:**
- **Tốt nhất**: beta = 0.7-1.0 cho Ackley Function
- Giá trị này cho phép đom đóm học hỏi từ nhau nhưng vẫn duy trì đủ diversity
- Với bài toán có nhiều local optima (như Ackley), beta vừa phải giúp tránh hội tụ sớm
- Nếu thuật toán hội tụ quá chậm, có thể tăng beta lên 1.0-1.5

### 3. Phân tích độ nhạy của tham số `gamma` cho Ackley

In [ ]:
# Test với các giá trị khác nhau của gamma
gamma_values_ackley = [0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
results_gamma_ackley = []

print("Đang test tham số gamma cho Ackley...")
for gamma in tqdm(gamma_values_ackley):
    result = run_fa_ackley(
        problem_num=problem_num_ackley,
        num_fireflies=20,
        beta=1.0,
        gamma=gamma,
        alpha=0.2,
        max_generations=max_generations_ackley,
        num_runs=num_runs_ackley,
    )
    result["gamma"] = gamma
    results_gamma_ackley.append(result)
    print(
        f"gamma={gamma}: Best={result['best_fitness']:.4f}, Avg={result['avg_fitness']:.4f}"
    )

In [ ]:
# Visualization cho gamma - Ackley
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Best Fitness vs gamma
ax1 = axes[0, 0]
gamma_vals_ackley = [r["gamma"] for r in results_gamma_ackley]
best_vals_ackley = [r["best_fitness"] for r in results_gamma_ackley]
avg_vals_ackley = [r["avg_fitness"] for r in results_gamma_ackley]
std_vals_ackley = [r["std_fitness"] for r in results_gamma_ackley]

ax1.semilogx(
    gamma_vals_ackley,
    best_vals_ackley,
    "co-",
    linewidth=2,
    markersize=8,
    label="Best Fitness",
)
ax1.semilogx(
    gamma_vals_ackley,
    avg_vals_ackley,
    "ys--",
    linewidth=2,
    markersize=8,
    label="Average Fitness",
)
ax1.fill_between(
    gamma_vals_ackley,
    np.array(avg_vals_ackley) - np.array(std_vals_ackley),
    np.array(avg_vals_ackley) + np.array(std_vals_ackley),
    alpha=0.3,
    color="yellow",
    label="Std Dev",
)
ax1.set_xlabel("Gamma (Hệ số hấp thụ ánh sáng)", fontsize=12, fontweight="bold")
ax1.set_ylabel("Fitness Value (Ackley)", fontsize=12, fontweight="bold")
ax1.set_title(
    "Ảnh hưởng của gamma đến hiệu suất (Ackley)", fontsize=14, fontweight="bold"
)
ax1.legend(loc="best")
ax1.grid(True, alpha=0.3)
ax1.set_yscale("log")

# 2. Convergence curves for different gamma
ax2 = axes[0, 1]
colors = plt.cm.cool(np.linspace(0, 1, len(results_gamma_ackley)))
for i, result in enumerate(results_gamma_ackley):
    avg_history = np.mean(result["convergence_history"], axis=0)
    ax2.plot(
        avg_history, color=colors[i], linewidth=2, label=f"gamma={result['gamma']}"
    )
ax2.set_xlabel("Generation", fontsize=12, fontweight="bold")
ax2.set_ylabel("Best Fitness (Ackley)", fontsize=12, fontweight="bold")
ax2.set_title(
    "Đường hội tụ với các giá trị gamma (Ackley)", fontsize=14, fontweight="bold"
)
ax2.legend(loc="best", fontsize=9)
ax2.grid(True, alpha=0.3)
ax2.set_yscale("log")

# 3. Box plot for variance analysis
ax3 = axes[1, 0]
data_for_boxplot_ackley = [r["all_best_fitnesses"] for r in results_gamma_ackley]
bp = ax3.boxplot(
    data_for_boxplot_ackley,
    labels=[f"{g}" for g in gamma_vals_ackley],
    patch_artist=True,
)
for patch, color in zip(bp["boxes"], colors):
    patch.set_facecolor(color)
ax3.set_xlabel("Gamma (Hệ số hấp thụ ánh sáng)", fontsize=12, fontweight="bold")
ax3.set_ylabel("Best Fitness Distribution (Ackley)", fontsize=12, fontweight="bold")
ax3.set_title("Phân phối fitness theo gamma (Ackley)", fontsize=14, fontweight="bold")
ax3.grid(True, alpha=0.3, axis="y")
ax3.tick_params(axis="x", rotation=45)
ax3.set_yscale("log")

# 4. Standard deviation analysis
ax4 = axes[1, 1]
ax4.plot(gamma_vals_ackley, std_vals_ackley, "ro-", linewidth=2, markersize=8)
ax4.set_xscale("log")
ax4.set_xlabel("Gamma (Hệ số hấp thụ ánh sáng)", fontsize=12, fontweight="bold")
ax4.set_ylabel("Standard Deviation", fontsize=12, fontweight="bold")
ax4.set_title("Độ ổn định theo gamma (Ackley)", fontsize=14, fontweight="bold")
ax4.grid(True, alpha=0.3)
ax4.set_yscale("log")

plt.tight_layout()
plt.show()

#### Kết luận cho tham số `gamma`

**Quan sát:**
- Gamma (hệ số hấp thụ ánh sáng) kiểm soát mức độ suy giảm attractiveness theo khoảng cách
- Gamma rất thấp (0.01-0.1): Tầm ảnh hưởng rộng, đom đóm bị ảnh hưởng bởi đom đóm xa
- Gamma trung bình (0.5-2.0): Tương tác cục bộ tốt, hỗ trợ tìm kiếm chi tiết
- Gamma cao (5.0-10.0): Tầm ảnh hưởng hẹp, đom đóm gần như tìm kiếm độc lập

**Khuyến nghị:**
- **Tốt nhất**: gamma = 0.5-1.0 cho Ackley Function
- Giá trị này tạo ra vùng ảnh hưởng vừa phải, cho phép exploration và exploitation cân bằng
- Gamma thấp hơn (0.1-0.5) phù hợp với không gian tìm kiếm lớn
- Gamma cao hơn (1.0-2.0) phù hợp khi cần intensification quanh vùng promising
- **Lưu ý**: Gamma và beta có tương tác mạnh, cần điều chỉnh đồng thời

### 4. Phân tích độ nhạy của tham số `alpha` cho Ackley

In [ ]:
# Test với các giá trị khác nhau của alpha
alpha_values_ackley = [0.05, 0.1, 0.2, 0.3, 0.5, 0.7, 1.0]
results_alpha_ackley = []

print("Đang test tham số alpha cho Ackley...")
for alpha in tqdm(alpha_values_ackley):
    result = run_fa_ackley(
        problem_num=problem_num_ackley,
        num_fireflies=20,
        beta=1.0,
        gamma=1.0,
        alpha=alpha,
        max_generations=max_generations_ackley,
        num_runs=num_runs_ackley,
    )
    result["alpha"] = alpha
    results_alpha_ackley.append(result)
    print(
        f"alpha={alpha}: Best={result['best_fitness']:.4f}, Avg={result['avg_fitness']:.4f}"
    )

In [ ]:
# Visualization cho alpha - Ackley
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Best Fitness vs alpha
ax1 = axes[0, 0]
alpha_vals_ackley = [r["alpha"] for r in results_alpha_ackley]
best_vals_ackley = [r["best_fitness"] for r in results_alpha_ackley]
avg_vals_ackley = [r["avg_fitness"] for r in results_alpha_ackley]
std_vals_ackley = [r["std_fitness"] for r in results_alpha_ackley]

ax1.plot(
    alpha_vals_ackley,
    best_vals_ackley,
    "ko-",
    linewidth=2,
    markersize=8,
    label="Best Fitness",
)
ax1.plot(
    alpha_vals_ackley,
    avg_vals_ackley,
    "bs--",
    linewidth=2,
    markersize=8,
    label="Average Fitness",
)
ax1.fill_between(
    alpha_vals_ackley,
    np.array(avg_vals_ackley) - np.array(std_vals_ackley),
    np.array(avg_vals_ackley) + np.array(std_vals_ackley),
    alpha=0.3,
    color="blue",
    label="Std Dev",
)
ax1.set_xlabel("Alpha (Tham số ngẫu nhiên hóa)", fontsize=12, fontweight="bold")
ax1.set_ylabel("Fitness Value (Ackley)", fontsize=12, fontweight="bold")
ax1.set_title(
    "Ảnh hưởng của alpha đến hiệu suất (Ackley)", fontsize=14, fontweight="bold"
)
ax1.legend(loc="best")
ax1.grid(True, alpha=0.3)
ax1.set_yscale("log")

# 2. Convergence curves for different alpha
ax2 = axes[0, 1]
colors = plt.cm.autumn(np.linspace(0, 1, len(results_alpha_ackley)))
for i, result in enumerate(results_alpha_ackley):
    avg_history = np.mean(result["convergence_history"], axis=0)
    ax2.plot(
        avg_history, color=colors[i], linewidth=2, label=f"alpha={result['alpha']}"
    )
ax2.set_xlabel("Generation", fontsize=12, fontweight="bold")
ax2.set_ylabel("Best Fitness (Ackley)", fontsize=12, fontweight="bold")
ax2.set_title(
    "Đường hội tụ với các giá trị alpha (Ackley)", fontsize=14, fontweight="bold"
)
ax2.legend(loc="best", fontsize=9)
ax2.grid(True, alpha=0.3)
ax2.set_yscale("log")

# 3. Box plot for variance analysis
ax3 = axes[1, 0]
data_for_boxplot_ackley = [r["all_best_fitnesses"] for r in results_alpha_ackley]
bp = ax3.boxplot(data_for_boxplot_ackley, labels=alpha_vals_ackley, patch_artist=True)
for patch, color in zip(bp["boxes"], colors):
    patch.set_facecolor(color)
ax3.set_xlabel("Alpha (Tham số ngẫu nhiên hóa)", fontsize=12, fontweight="bold")
ax3.set_ylabel("Best Fitness Distribution (Ackley)", fontsize=12, fontweight="bold")
ax3.set_title("Phân phối fitness theo alpha (Ackley)", fontsize=14, fontweight="bold")
ax3.grid(True, alpha=0.3, axis="y")
ax3.set_yscale("log")

# 4. Exploration vs Exploitation trade-off
ax4 = axes[1, 1]
# Tính độ thay đổi trung bình của fitness qua các thế hệ (đo lường exploration)
exploration_metric_ackley = []
for result in results_alpha_ackley:
    avg_history = np.mean(result["convergence_history"], axis=0)
    # Tính tổng sự thay đổi tuyệt đối
    changes = np.abs(np.diff(avg_history))
    exploration_metric_ackley.append(np.mean(changes))

ax4.plot(
    alpha_vals_ackley, exploration_metric_ackley, "go-", linewidth=2, markersize=10
)
ax4.set_xlabel("Alpha (Tham số ngẫu nhiên hóa)", fontsize=12, fontweight="bold")
ax4.set_ylabel("Mức độ exploration (avg change)", fontsize=12, fontweight="bold")
ax4.set_title(
    "Khả năng exploration theo alpha (Ackley)", fontsize=14, fontweight="bold"
)
ax4.grid(True, alpha=0.3)
ax4.set_yscale("log")

plt.tight_layout()
plt.show()

#### Kết luận cho tham số `alpha`

**Quan sát:**
- Alpha (tham số ngẫu nhiên hóa) kiểm soát mức độ random walk trong chuyển động
- Alpha thấp (0.05-0.1): Ít nhiễu, hội tụ nhanh nhưng dễ bị trapped
- Alpha trung bình (0.2-0.3): Cân bằng tốt giữa exploitation và exploration
- Alpha cao (0.5-1.0): Nhiều nhiễu, tăng exploration nhưng hội tụ chậm

**Khuyến nghị:**
- **Tốt nhất**: alpha = 0.2-0.3 cho Ackley Function
- Giá trị này cho phép thuật toán thoát khỏi local optima mà vẫn hội tụ ổn định
- Có thể sử dụng alpha thích ứng: bắt đầu cao (0.5) để exploration, giảm dần về (0.1) để exploitation
- **Chiến lược đề xuất**: `alpha(t) = alpha_0 * exp(-t/T)` với alpha_0=0.5, T=max_generations
- Với bài toán đơn giản hơn, có thể giảm alpha xuống 0.1-0.15

### 5. Phân tích tổng hợp (Heatmap) - Tương tác giữa beta và gamma cho Ackley

In [ ]:
# Phân tích tương tác giữa beta và gamma cho Ackley
beta_test_ackley = [0.3, 0.7, 1.0, 1.5, 2.0]
gamma_test_ackley = [0.1, 0.5, 1.0, 2.0, 5.0]

heatmap_data_ackley = np.zeros((len(beta_test_ackley), len(gamma_test_ackley)))

print("Đang test tương tác beta-gamma cho Ackley...")
for i, beta in enumerate(tqdm(beta_test_ackley)):
    for j, gamma in enumerate(gamma_test_ackley):
        result = run_fa_ackley(
            problem_num=problem_num_ackley,
            num_fireflies=20,
            beta=beta,
            gamma=gamma,
            alpha=0.2,
            max_generations=max_generations_ackley,
            num_runs=3,  # Giảm số lần chạy để tăng tốc
        )
        heatmap_data_ackley[i, j] = result["avg_fitness"]
        print(f"beta={beta}, gamma={gamma}: Avg={result['avg_fitness']:.4f}")

In [ ]:
# Visualization heatmap cho tương tác beta-gamma - Ackley
fig, ax = plt.subplots(figsize=(12, 8))

# Sử dụng log scale cho colormap để hiển thị tốt hơn
import matplotlib.colors as colors

im = ax.imshow(
    heatmap_data_ackley,
    cmap="YlOrRd_r",
    aspect="auto",
    norm=colors.LogNorm(vmin=heatmap_data_ackley.min(), vmax=heatmap_data_ackley.max()),
)

# Thiết lập ticks và labels
ax.set_xticks(np.arange(len(gamma_test_ackley)))
ax.set_yticks(np.arange(len(beta_test_ackley)))
ax.set_xticklabels(gamma_test_ackley)
ax.set_yticklabels(beta_test_ackley)

# Thêm colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label("Average Fitness (Ackley - log scale)", fontsize=12, fontweight="bold")

# Thêm annotations
for i in range(len(beta_test_ackley)):
    for j in range(len(gamma_test_ackley)):
        text = ax.text(
            j,
            i,
            f"{heatmap_data_ackley[i, j]:.3f}",
            ha="center",
            va="center",
            color="black",
            fontsize=10,
        )

ax.set_xlabel("Gamma (Hệ số hấp thụ ánh sáng)", fontsize=12, fontweight="bold")
ax.set_ylabel("Beta (Hệ số hấp dẫn)", fontsize=12, fontweight="bold")
ax.set_title(
    "Tương tác giữa Beta và Gamma - Heatmap của Average Fitness (Ackley)",
    fontsize=14,
    fontweight="bold",
)

plt.tight_layout()
plt.show()

#### Kết luận về tương tác Beta-Gamma

**Quan sát từ Heatmap:**
- Beta và gamma có mối quan hệ tương tác mạnh với nhau
- Vùng tối ưu (màu sáng trên heatmap): beta=0.7-1.0, gamma=0.5-1.0
- Beta cao + Gamma cao: Hội tụ rất nhanh nhưng kém chất lượng (dễ bị local optima)
- Beta thấp + Gamma thấp: Exploration tốt nhưng hội tụ chậm
- **Sweet spot**: Sự kết hợp cân bằng tạo ra hiệu suất tốt nhất

**Khuyến nghị tổng hợp:**
- Với Ackley Function, nên chọn: **beta=0.7-1.0 và gamma=0.5-1.0**
- Nếu dimension cao (>20): tăng gamma lên 1.0-2.0 để tăng cường local search
- Nếu dimension thấp (<10): giảm gamma xuống 0.3-0.5 để tăng global search
- **Nguyên tắc**: beta * gamma nên trong khoảng 0.5-1.5 để có hiệu suất tốt

---

## So sánh: Knapsack vs Ackley Function

### Đặc điểm bài toán

| Tiêu chí | Knapsack | Ackley Function |
|----------|----------|-----------------|
| **Loại bài toán** | Combinatorial (Rời rạc) | Continuous (Liên tục) |
| **Không gian tìm kiếm** | Binary {0,1}ⁿ | Real-valued ℝⁿ |
| **Mục tiêu** | Maximization | Minimization |
| **Local optima** | Nhiều (exponential) | Rất nhiều (highly multimodal) |
| **Độ phức tạp** | NP-Hard | Highly non-linear |

### Tham số tối ưu cho Firefly Algorithm

| Tham số | Knapsack | Ackley | Nhận xét |
|---------|----------|--------|----------|
| **num_fireflies** | 20-30 | 20-30 | ✅ Tương tự cho cả 2 |
| **beta** | 1.0-1.5 | 0.7-1.0 | ⚠️ Knapsack cần cao hơn (hội tụ mạnh) |
| **gamma** | 1.0-2.0 | 0.5-1.0 | ⚠️ Knapsack cần cao hơn (local search) |
| **alpha** | 0.2-0.3 | 0.2-0.3 | ✅ Tương tự cho cả 2 |

### Điểm khác biệt chính

**🎯 Knapsack Problem:**
- Cần **beta cao hơn** (1.0-1.5) để tăng cường exploitation trong không gian rời rạc
- Cần **gamma cao hơn** (1.0-2.0) để tập trung vào local search với các bit lân cận
- Bài toán tối ưu tổ hợp → cần hội tụ nhanh về vùng promising

**🔬 Ackley Function:**
- Cần **beta thấp hơn** (0.7-1.0) để tránh premature convergence do có quá nhiều local optima
- Cần **gamma thấp hơn** (0.5-1.0) để duy trì exploration trong không gian liên tục
- Bài toán multimodal → cần cân bằng exploration/exploitation tốt hơn

### Kết luận chung

✨ Firefly Algorithm thể hiện **tính linh hoạt cao** khi áp dụng cho cả bài toán rời rạc (Knapsack) và liên tục (Ackley), chỉ cần điều chỉnh tham số phù hợp với đặc thêm từng loại bài toán.